In [ ]:
import pandas as pd
import sqlite3

# Merging and Joining

# Inner Join on Chinook Database
conn = sqlite3.connect('chinook.db')
customers = pd.read_sql_query("SELECT * FROM customers", conn)
invoices = pd.read_sql_query("SELECT * FROM invoices", conn)
conn.close()

# Inner join between customers and invoices
customer_invoices = pd.merge(customers, invoices, on='CustomerId', how='inner')

# Count of invoices per customer
invoice_counts = customer_invoices.groupby('CustomerId')['InvoiceId'].count().reset_index()
print(invoice_counts)

# Outer Join on Movie Data
movie_df = pd.read_csv('movie.csv')

# Creating two smaller DataFrames
movie_df1 = movie_df[['director_name', 'color']]
movie_df2 = movie_df[['director_name', 'num_critic_for_reviews']]

# Left join and full outer join
left_join = pd.merge(movie_df1, movie_df2, on='director_name', how='left')
full_outer_join = pd.merge(movie_df1, movie_df2, on='director_name', how='outer')

print(f"Left Join rows: {left_join.shape[0]}")
print(f"Full Outer Join rows: {full_outer_join.shape[0]}")

In [ ]:
# Grouping and Aggregating

# Grouped Aggregations on Titanic
titanic_df = pd.read_excel('titanic.xlsx')
pclass_group = titanic_df.groupby('Pclass').agg({
    'Age': 'mean',
    'Fare': 'sum',
    'PassengerId': 'count'
}).reset_index()
print(pclass_group)

# Multi-level Grouping on Movie Data
movie_group = movie_df.groupby(['color', 'director_name']).agg({
    'num_critic_for_reviews': 'sum',
    'duration': 'mean'
}).reset_index()
print(movie_group)

# Nested Grouping on Flights
flights_df = pd.read_parquet('flights.parquet')
flights_group = flights_df.groupby(['Year', 'Month']).agg({
    'FlightNum': 'count',
    'ArrDelay': 'mean',
    'DepDelay': 'max'
}).reset_index()
print(flights_group)


In [ ]:
# Applying Functions

# Apply a Custom Function on Titanic
def age_group(age):
    return 'Child' if age < 18 else 'Adult'

titanic_df['Age_Group'] = titanic_df['Age'].apply(age_group)
print(titanic_df[['Age', 'Age_Group']].head())

# Normalize Employee Salaries
employee_df = pd.read_csv('employee.csv')

# Normalizing salaries by department
employee_df['Normalized_Salary'] = employee_df.groupby('Department')['Salary'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)
print(employee_df.head())

# Custom Function on Movies
def classify_duration(duration):
    if duration < 60:
        return 'Short'
    elif duration <= 120:
        return 'Medium'
    else:
        return 'Long'

movie_df['Duration_Category'] = movie_df['duration'].apply(classify_duration)
print(movie_df[['title', 'duration', 'Duration_Category']].head())

# Using pipe

# Pipeline on Titanic
def filter_survived(df):
    return df[df['Survived'] == 1]

def fill_missing_age(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    return df

def fare_per_age(df):
    df['Fare_Per_Age'] = df['Fare'] / df['Age']
    return df

titanic_pipeline = (titanic_df.pipe(filter_survived)
                                .pipe(fill_missing_age)
                                .pipe(fare_per_age))
print(titanic_pipeline.head())

# Pipeline on Flights
def filter_delayed_flights(df):
    return df[df['DepDelay'] > 30]

def calculate_delay_per_hour(df):
    df['Delay_Per_Hour'] = df['DepDelay'] / df['AirTime']
    return df

flights_pipeline = (flights_df.pipe(filter_delayed_flights)
                               .pipe(calculate_delay_per_hour))
print(flights_pipeline.head())